# 08 — Estimator Selection and Debugging

We show a diagnostics-driven estimator selection workflow and a debugging
checklist when overlap or model fit is poor.

## Setup

```
pip install "causalrl[plots]"
```

In [ ]:
from __future__ import annotations

import numpy as np

from crl.assumptions import AssumptionSet
from crl.assumptions_catalog import MARKOV, OVERLAP, SEQUENTIAL_IGNORABILITY
from crl.benchmarks.mdp_synth import SyntheticMDP, SyntheticMDPConfig
from crl.estimands.policy_value import PolicyValueEstimand
from crl.selectors import select_estimator
from crl.utils.seeding import set_seed

In [ ]:
set_seed(0)
np.random.seed(0)

## Run selection

We use a heuristic score that favors stable importance weights and reasonable
model fit.

In [ ]:
benchmark = SyntheticMDP(SyntheticMDPConfig(seed=0, horizon=5))
dataset = benchmark.sample(num_trajectories=200, seed=1)

estimand = PolicyValueEstimand(
    policy=benchmark.target_policy,
    discount=dataset.discount,
    horizon=dataset.horizon,
    assumptions=AssumptionSet([SEQUENTIAL_IGNORABILITY, OVERLAP, MARKOV]),
)

best = select_estimator(
    dataset,
    estimand,
    candidates=["is", "wis", "pdis", "dr", "wdr", "mrdr", "fqe"],
)
best

## Debug playbook

- **Overlap bad** → inspect ESS and weight tails, consider WIS/DR, or collect
  more coverage.
- **Model fit bad** → check Q-model MSE, increase model capacity, or switch to
  IS-based estimators.
- **Propensities unknown** → estimate behavior policy or use model-based OPE.

## Takeaways

- Estimator selection is heuristic, but diagnostics make it principled.
- Always triangulate with multiple estimators and failure-mode checks.